In [1]:
import sys

sys.path.append("../sketchformer")
sys.path.append("../src")


In [2]:
from isketcher import InteractiveSketcher
import tensorflow as tf
import numpy as np
import pandas as pd
import random


In [3]:
num_layers = 8
d_model = 64
dff = 128
num_heads = 4
dropout_rate = 0.1
target_object_num = 41


class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)


In [4]:
# create model

interactive_sketcher = InteractiveSketcher(
    num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff,
    object_num=target_object_num, rate=dropout_rate)

# restore model
checkpoint_path = "./models/demo"

ckpt = tf.train.Checkpoint(epoch=tf.Variable(1),
                           transformer=interactive_sketcher,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(
    ckpt, checkpoint_path, max_to_keep=5)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print('Latest checkpoint restored!!')


Latest checkpoint restored!!


In [5]:
dataset = np.load('../data/isketcher/dataset.npz')
x_test, y_test = dataset['x_test'], dataset['y_test']
print(x_test.shape)
print(y_test.shape)


(1113, 43, 130)
(1113, 43)


In [6]:
# class label
df = pd.read_csv('../outputs/sketchyscene_quickdraw.csv')
df = df.dropna(subset=['quickdraw_label'])
class_names = ['none']
for row in df.itertuples():
    class_names.append(row.quickdraw_label)
class_to_num = dict(zip(class_names, range(0, len(class_names))))

print(class_names)
print(len(class_names))
print(class_to_num)


['none', 'airplane', 'apple', 'hot air balloon', 'banana', 'basket', 'bee', 'bench', 'bicycle', 'bird', 'wine bottle', 'bucket', 'bus', 'butterfly', 'car', 'cat', 'chair', 'cloud', 'cow', 'cup', 'dog', 'duck', 'fence', 'flower', 'grapes', 'grass', 'horse', 'house', 'moon', 'mountain', 'face', 'pig', 'rabbit', 'sheep', 'star', 'streetlight', 'sun', 'table', 'tree', 'truck', 'umbrella']
41
{'none': 0, 'airplane': 1, 'apple': 2, 'hot air balloon': 3, 'banana': 4, 'basket': 5, 'bee': 6, 'bench': 7, 'bicycle': 8, 'bird': 9, 'wine bottle': 10, 'bucket': 11, 'bus': 12, 'butterfly': 13, 'car': 14, 'cat': 15, 'chair': 16, 'cloud': 17, 'cow': 18, 'cup': 19, 'dog': 20, 'duck': 21, 'fence': 22, 'flower': 23, 'grapes': 24, 'grass': 25, 'horse': 26, 'house': 27, 'moon': 28, 'mountain': 29, 'face': 30, 'pig': 31, 'rabbit': 32, 'sheep': 33, 'star': 34, 'streetlight': 35, 'sun': 36, 'table': 37, 'tree': 38, 'truck': 39, 'umbrella': 40}


In [41]:
# test

scene_index = 2
print(x_test[scene_index].shape)

print("----", 0, "-------")
print("real label: ", class_names[int(y_test[scene_index, 0])])
print("real position: ", x_test[scene_index, 0, -2:] * 750)

for i in range(1, len(x_test[scene_index])):

    c_out, p_out, _ = interactive_sketcher(
        x_test[scene_index, :i], training=False, look_ahead_mask=None)

    c_real_id = int(y_test[scene_index, i])
    if c_real_id == 0:
        break
    p_real = x_test[scene_index, i, -2:]

    c_pred = c_out[0, -1, :]  # 最後のスケッチを取得
    c_pred_id = tf.argmax(c_pred, axis=-1)
    p_pred = p_out[0, -1, :]  # 最後のスケッチを取得

    print("----", i, "-------")
    print("real label: ", class_names[c_real_id])
    print("pred label: ", class_names[c_pred_id])

    print("real position: ", p_real * 750)
    print("pred position: ", p_pred.numpy() * 750)


(43, 130)
---- 0 -------
real label:  cloud
real position:  [484.99997  65.     ]
---- 1 -------
real label:  cloud
pred label:  cloud
real position:  [300.  70.]
pred position:  [367.67947 193.48589]
---- 2 -------
real label:  cloud
pred label:  cloud
real position:  [80. 70.]
pred position:  [367.8154  192.72011]
---- 3 -------
real label:  cloud
pred label:  cloud
real position:  [565.  75.]
pred position:  [367.7949  192.77812]
---- 4 -------
real label:  cloud
pred label:  cloud
real position:  [135.  85.]
pred position:  [367.8534  194.04968]
---- 5 -------
real label:  mountain
pred label:  cloud
real position:  [115. 125.]
pred position:  [367.533   193.61221]
---- 6 -------
real label:  mountain
pred label:  mountain
real position:  [439.99997 145.     ]
pred position:  [369.5006  312.55164]
---- 7 -------
real label:  mountain
pred label:  mountain
real position:  [625. 125.]
pred position:  [368.16507 309.63608]
---- 8 -------
real label:  mountain
pred label:  mountain
rea

In [42]:
# draw canvas

from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

canvas = Image.new('RGB', (750, 750), (255, 255, 255))

scene_index = 2
pred_index = 2

draw = ImageDraw.Draw(canvas)
r = 5
font = ImageFont.truetype("/usr/share/fonts/truetype/ubuntu/Ubuntu-R.ttf", size=20)

for i in range(pred_index):
    label = class_names[int(y_test[scene_index, i])]
    x, y = x_test[scene_index, i, -2:] * 750
    draw.ellipse((x - r, y - r, x + r, y + r), fill=(0, 0, 0))
    draw.text((x + r, y + r), label, fill='black', font=font)

c_out, p_out, _ = interactive_sketcher(
    x_test[scene_index, :pred_index], training=False, look_ahead_mask=None)

c_pred = c_out[0, -1, :]  # 最後のスケッチを取得
c_pred_id = tf.argmax(c_pred, axis=-1)
x_pred, y_pred = p_out[0, -1, :] * 750  # 最後のスケッチを取得

next_canvas = canvas.copy()
next_draw = ImageDraw.Draw(next_canvas)
next_draw.ellipse((x_pred - r, y_pred - r, x_pred +
                  r, y_pred + r), fill=(0, 0, 0))
next_draw.text((x_pred + r, y_pred + r), class_names[c_pred_id], fill='black', font=font)

canvas.save('canvas.png')
next_canvas.save('next_canvas.png')